In [1]:
import os
import pandas as pd
import pickle
import ast
import sys
import re
import numpy as np

In [2]:
OUT_DIR="/media/SSD/paper_data/"

In [3]:
accuracies_single = pd.read_csv(OUT_DIR+"/sakura/accuracies/accuracies_single_sensors_all.csv", index_col=None)
accuracies_all = pd.read_csv(OUT_DIR+"/sakura/accuracies/accuracies_kfold.csv", index_col=None)

In [4]:
sensor_tag="all"
seeds = [0]
preprocessings = ["none"]
models = ["resnet"]
sensors = [0, 1, 2, 3, 4]
datasets = ["IN/nops", "OUT1/nops", "OUT1/random", "OUT2/nops"]
i = 10000
topks = [2, 3, 4, 5, 6]
topk_empty = []

columns = ['dataset', 'model', 'type', 'n_sensors', 'label', 'preprocessing', 'seed', 'kfold', 'accuracy']
accuracies = pd.DataFrame(columns=columns)

jobs_to_repeat = []

results_idx = 0

for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                
                    model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    worst = list(accuracies_single.loc[(accuracies_single.kfold=='avg') & (accuracies_single.dataset=="Exp-"+dataset_small) & (accuracies_single.model=='resnet') & (accuracies_single.seed==0 & (accuracies_single.preprocessing==preprocessing))].sort_values(['accuracy'])['sensor'])
                    best = list(reversed(worst))
                    
                    for case in ['best', 'worst']:
                        for s_no in range(1, 4):
                            sensor_ids = []
                            for ids in range(0, s_no+1):
                                if(case=='best'):
                                    sensor_ids.append(best[ids])
                                else:
                                    sensor_ids.append(worst[ids])
                            sensor_ids_original = sensor_ids.copy()
                            sensor_ids.sort()
                    
                            sensor_names = ""
                            for ids in sensor_ids:
                                sensor_names+=(str(ids)+"-")
                            
                            if(dataset_small=="out1-nops" or dataset_small=="out1-random"):
                                offset = 5
                            elif(dataset_small=="out2-nops"):
                                offset = 10
                            else:
                                offset = 0
                            sensor_names_label = ""
                            for ids in sensor_ids_original:
                                sensor_names_label+=("S"+str(ids+offset)+",")
                            sensor_names_label = sensor_names_label[:-1]
                                
                            acc_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensor-"+sensor_names+case+"_10fold/out/kfold_results.txt"

                            # Check if job was completed
                            if(os.path.exists(acc_file)!=True):
                                print("Results for "+acc_file+" not yet ready!")
                                for kfold_tmp in range(10):
                                    accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, case, s_no+1, sensor_names_label, preprocessing, 0, kfold_tmp, "-"]
                                    results_idx = results_idx+1
                                accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, case, s_no+1, sensor_names_label, preprocessing, 0, "avg", "-"]
                                results_idx = results_idx+1
                                continue

                            # Check if data was stored safetly
                            num_lines = sum(1 for line in open(acc_file))
                            if(num_lines <= 11):
                                print("Missing data for "+"Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensor-"+str(sensor)+"_10fold")
                                jobs_to_repeat.append("prs-"+dataset_small+"-"+model_small+"-p"+preprocessing_small+"-"+str(i)+"-s"+str(seed)+"-sensor-"+str(sensor))
                                for kfold_tmp in range(10):
                                    accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, case, s_no+1, sensor_names_label, preprocessing, 0, kfold_tmp, "-"]
                                    results_idx = results_idx+1
                                accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, case, s_no+1, sensor_names_label, preprocessing, 0, "avg", "-"]
                                results_idx = results_idx+1
                                continue
                            #print(out_path)

                            file1 = open(acc_file, 'r')
                            Lines = file1.readlines()

                            for count, line in enumerate(Lines):
                                #print("Line{}: {}".format(count, line.strip()))
                                if(count>=1 and count <=10):
                                    kf_accuracy = re.findall("\d+\.\d+", line.strip())[1]
                                    #row = pd.DataFrame(['Exp-IN', 'LSTM+CNN', 'none', 0, count, kf_accuracy])
                                    #accuracies = pd.concat([accuracies, row], axis=0, ignore_index=True)
                                    accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, case, s_no+1, sensor_names_label, preprocessing, 0, count-1, kf_accuracy]
                                    results_idx = results_idx+1
                                if(count==12):
                                    kf_accuracy = re.findall("\d+\.\d+", line.strip())[0]
                                    accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, case, s_no+1, sensor_names_label, preprocessing, 0, "avg", kf_accuracy]
                                    results_idx = results_idx+1

                            file1.close()
                    


# Add best and worst single sensors
for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                
                    model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    worst = list(accuracies_single.loc[(accuracies_single.kfold=='avg') & (accuracies_single.dataset=="Exp-"+dataset_small) & (accuracies_single.model=='resnet') & (accuracies_single.seed==0 & (accuracies_single.preprocessing==preprocessing))].sort_values(['accuracy'])['sensor'])
                    best = list(reversed(worst))
                    
                    if(dataset_small=="out1-nops" or dataset_small=="out1-random"):
                        offset = 5
                    elif(dataset_small=="out2-nops"):
                        offset = 10
                    else:
                        offset = 0
                    
                    for case in ['best', 'worst']:
                        s_no = 0                     
                        if(case=='best'):
                            sensor_current = best[s_no]
                        else:
                            sensor_current = worst[s_no]
                        for kf in range(0, 10):
                            acc = accuracies_single.loc[(accuracies_single.kfold==str(kf)) & (accuracies_single.dataset=="Exp-"+dataset_small) & (accuracies_single.model=='resnet') & (accuracies_single.seed==0 & (accuracies_single.preprocessing==preprocessing)) & (accuracies_single.sensor==sensor_current)].loc[:, 'accuracy'].iloc[0]
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, case, s_no+1, "S"+str(sensor_current+offset), preprocessing, 0, kf, acc]
                            results_idx = results_idx+1
                        acc = accuracies_single.loc[(accuracies_single.kfold=='avg') & (accuracies_single.dataset=="Exp-"+dataset_small) & (accuracies_single.model=='resnet') & (accuracies_single.seed==0 & (accuracies_single.preprocessing==preprocessing)) & (accuracies_single.sensor==sensor_current)].loc[:, 'accuracy'].iloc[0]
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, case, s_no+1, "S"+str(sensor_current+offset), preprocessing, 0, 'avg', acc]
                        results_idx = results_idx+1
                    
# Add all sensors
for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                
                model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                worst = list(accuracies_single.loc[(accuracies_single.kfold=='avg') & (accuracies_single.dataset=="Exp-"+dataset_small) & (accuracies_single.model=='resnet') & (accuracies_single.seed==0 & (accuracies_single.preprocessing==preprocessing))].sort_values(['accuracy'])['sensor'])
                best = list(reversed(worst))
                
                #if(dataset_small=="out1-nops" or dataset_small=="out1-random"):
                #    sensor_ids = "S5,S6,S7,S8,S9"
                #elif(dataset_small=="out2-nops"):
                #    sensor_ids = "S10,S11,S12,S13,S14"
                #else:
                #    sensor_ids = "S0,S1,S2,S3,S4"
                    

                
                for case in ['best', 'worst']:
                    
                    sensor_ids = []
                    for ids in range(0, 5):
                        if(case=='best'):
                            sensor_ids.append(best[ids])
                        else:
                            sensor_ids.append(worst[ids])
                    sensor_ids_original = sensor_ids.copy()

                    if(dataset_small=="out1-nops" or dataset_small=="out1-random"):
                        offset = 5
                    elif(dataset_small=="out2-nops"):
                        offset = 10
                    else:
                        offset = 0
                    sensor_names_label = ""
                    for ids in sensor_ids_original:
                        sensor_names_label+=("S"+str(ids+offset)+",")
                    sensor_names_label = sensor_names_label[:-1]
                    
                    for kf in range(0, 10):
                        acc = accuracies_all.loc[(accuracies_all.kfold==str(kf)) & (accuracies_all.dataset=="Exp-"+dataset_small) & (accuracies_all.model=='resnet') & (accuracies_all.seed==0 & (accuracies_all.preprocessing==preprocessing))].loc[:, 'accuracy'].iloc[0]
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, case, 5, sensor_names_label, preprocessing, 0, kf, acc]
                        results_idx = results_idx+1
                    acc = accuracies_all.loc[(accuracies_all.kfold=='avg') & (accuracies_all.dataset=="Exp-"+dataset_small) & (accuracies_all.model=='resnet') & (accuracies_all.seed==0 & (accuracies_all.preprocessing==preprocessing))].loc[:, 'accuracy'].iloc[0]
                    accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, case, 5, sensor_names_label, preprocessing, 0, 'avg', acc]
                    results_idx = results_idx+1


accuracies['accuracy'] = accuracies['accuracy'].astype(float)

In [10]:
accuracies.to_csv(OUT_DIR+"/sakura/accuracies/accuracies_incremental_all.csv", index=False)
accuracies.iloc[:, 0:9].loc[accuracies.kfold=='avg'].reset_index(drop=True).to_csv(OUT_DIR+"/sakura/accuracies/accuracies_incremental_averaged.csv", index=False)

In [6]:
accuracies_single[accuracies_single.kfold=='avg']

,dataset,model,sensor,preprocessing,seed,kfold,accuracy
10,Exp-in-nops,resnet,0,none,0,avg,48.759729
21,Exp-in-nops,resnet,1,none,0,avg,60.224054
32,Exp-in-nops,resnet,2,none,0,avg,64.586217
43,Exp-in-nops,resnet,3,none,0,avg,71.523243
54,Exp-in-nops,resnet,4,none,0,avg,53.144864
65,Exp-out1-nops,resnet,0,none,0,avg,39.320540
76,Exp-out1-nops,resnet,1,none,0,avg,40.524054
87,Exp-out1-nops,resnet,2,none,0,avg,55.502703
98,Exp-out1-nops,resnet,3,none,0,avg,45.000270
109,Exp-out1-nops,resnet,4,none,0,avg,44.344054
